# Overview

In this notebook we are going to look at running scikit-learn models against an Apache Spark cluster. Unlike the models packaged with Apache Spark, scikit-learn models are not built to be distributed and thus cannot parallelize calculations involved to train the model.

It assumes you are already familiar with running spark and have read the following notebooks:
- [Install Apache Spark Prerequisites](Install%20Apache%20Spark%20Prerequisites.ipynb)
- [Spark Pi - The Hello World Example For Apache spark](Spark%20Pi%20-%20The%20Hello%20World%20Example%20For%20Apache%20spark.ipynb)
- [Intro To Koalas](Intro%20To%20Koalas.ipynb)
- <a href="../Cluster%20Analysis/K-Means.ipynb">Cluster Analysis/K-Means</a>

## Adjenda
1. Create SparkContext
2. Create Web Server To Host Data
3. Load The Data
8. Prepare Worker Nodes
9. Submit Python Code To Spark Cluster
10. Cleanup Spark and Kubernetes

# 1. Create SparkContext

In [1]:
# Load a helper module
import importlib.util
spec = importlib.util.spec_from_file_location("spark_helper", "../../../Utilities/spark_helper.py")
spark_helper = importlib.util.module_from_spec(spec)
spec.loader.exec_module(spark_helper)

In [2]:
spark_app_name = "spark-jupyter-sklearn"
docker_image = "tschneider/apache-spark-k8:v7"
k8_master_ip = "15.4.7.11"
spark_session = spark_helper.create_spark_session(spark_app_name, docker_image, k8_master_ip)
sc = spark_session.sparkContext

Setting SPARK_HOME
/usr/lib/spark-3.1.1-bin-hadoop2.7

Running findspark.init() function
['/usr/lib/spark-3.1.1-bin-hadoop2.7/python', '/usr/lib/spark-3.1.1-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip', '/ml-training-jupyter-notebooks/Machine Learning/Big Data And Big Compute/Apache Spark', '/usr/local/lib/python39.zip', '/usr/local/lib/python3.9', '/usr/local/lib/python3.9/lib-dynload', '', '/usr/local/lib/python3.9/site-packages']

Setting PYSPARK_PYTHON
/usr/local/bin/python3

Configuring URL for kubernetes master
k8s://https://15.4.7.11:6443

Determining IP Of Server
The ip was detected as: 15.4.12.12

Creating SparkConf Object
('spark.master', 'k8s://https://15.4.7.11:6443')
('spark.app.name', 'spark-jupyter-sklearn')
('spark.submit.deploy.mode', 'cluster')
('spark.kubernetes.container.image', 'tschneider/apache-spark-k8:v7')
('spark.kubernetes.namespace', 'spark')
('spark.kubernetes.pyspark.pythonVersion', '3')
('spark.kubernetes.authenticate.driver.serviceAccountName', 'spark-s

22/01/15 21:32:02 WARN Utils: Your hostname, localhost.localdomain resolves to a loopback address: 127.0.0.1; using 15.4.12.12 instead (on interface eth0)



Done!


We can look at kubernetes to see that out worker nodes were created. 

The first time we create the spark context with a given docker image, the image will need to be downloaded (which takes some time). As a result, we may see the pods with a status of "ContainerCreating". In this case, we will need to wait until the containers are in a "Running" state.

```
kubectl -n spark get pod
NAME                                        READY   STATUS              RESTARTS   AGE
spark-jupyter-win-3ed7f27984f7563a-exec-1   0/1     ContainerCreating   0          12m
spark-jupyter-win-3ed7f27984f7563a-exec-2   0/1     ContainerCreating   0          12m
spark-jupyter-win-3ed7f27984f7563a-exec-3   0/1     ContainerCreating   0          12m
```

We can check the status of the docker pull by logging into the container and running the docker pull command to attach to the running process:
```
kubectl -n spark exec -ti docker pull tschneider/pyspark:v3 docker pull tschneider/pyspark:v4
v3: Pulling from tschneider/pyspark
2d473b07cdd5: Already exists
71d236fb1195: Already exists
2e22160d8cab: Already exists
e99d962ac218: Pull complete
Digest: sha256:eb74701b4ae909c40046ff68b1044b09b11895e175c955dfd8afe9fe680309cf
Status: Downloaded newer image for tschneider/pyspark:v3
docker.io/tschneider/pyspark:v3
[root@os004k8-worker002 ~]# docker pull tschneider/pyspark:v4
v4: Pulling from tschneider/pyspark
2d473b07cdd5: Already exists
71d236fb1195: Already exists
2e22160d8cab: Already exists
c556a717fe5d: Downloading [=======================>                           ]  578.7MB/1.246GB
```

In [3]:
! kubectl -n spark get pod

NAME                                            READY     STATUS    RESTARTS   AGE
spark-jupyter-sklearn-03a4877e5fa79c97-exec-1   1/1       Running   0          21s
spark-jupyter-sklearn-03a4877e5fa79c97-exec-2   1/1       Running   0          21s
spark-jupyter-sklearn-03a4877e5fa79c97-exec-3   1/1       Running   0          21s


# 2. Setup Datastore

In [4]:
import os

csv_file_name = "nasdaq_2019.csv"
csv_relative_file_path = "../../../Example Data Sets/{0}".format(csv_file_name)
csv_absolute_file_path = os.path.abspath(csv_relative_file_path)
csv_link_path = "/{0}".format(csv_file_name)

if os.path.exists(csv_link_path):
    if os.path.islink(csv_link_path):
        print("Symlink exists at {0}".format(csv_link_path))
    elif os.path.isfile(file):
        print("File exists at {0}".format(csv_link_path))
    else:
        raise Exception("Something is wrong. An object exists where we want to create a symlink.")
else:
    os.symlink(csv_absolute_file_path, csv_link_path)
    print("Symlink created as {0} -> {1}".format(csv_link_path, csv_absolute_file_path))

Symlink exists at /nasdaq_2019.csv


# 3. Create web server to host data

## 3.1. Determine the current working directory. 

Note: There is a trick to doing this inside a jupyter notebook and so we will use a special library to get that information.

In [5]:
import pyprojroot
project_root_dir  = pyprojroot.here()
print(project_root_dir)

/ml-training-jupyter-notebooks


## 3.2. Load the module for the webserver from our utilities directory

In [6]:
# Import the module for the web server we wrote
import importlib.util
spec = importlib.util.spec_from_file_location("PythonHttpFileServer", "../../../Utilities/PythonHttpFileServer.py")
PythonHttpFileServer = importlib.util.module_from_spec(spec)
spec.loader.exec_module(PythonHttpFileServer)

## 3.3. Configure logging

In [7]:
# Configure the logger and log level
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

## 3.4. Start the webserver in a new thread

In [8]:
import os

data_sub_dir = "Example Data Sets"
web_root = os.path.join(project_root_dir, data_sub_dir)

if not os.path.exists(web_root):
    raise Exception("The web root for the server does not exist.")

In [9]:
# Start the webserver in a thread so the cell is not stuck in a running state
import threading

var_exists = 'web_server_thread' in locals() or 'web_server_thread' in globals()
if not var_exists:
    web_server_port = 80
    web_server_args = (web_server_port, web_root)
    web_server_thread = threading.Thread(target=PythonHttpFileServer.run_server, args=web_server_args)
    web_server_thread.start()
else:
    print("Web Server thread already exists")
    print("To kill it you need to restart the kernel.")

INFO:root:Starting server on port 80
INFO:root:Web root specified as: /ml-training-jupyter-notebooks/Example Data Sets


# 4. Load The Data

Loading the data is not as intuitive as one would think. We Instruct the spark cluster to download a file from the web server. But when we do this, the file is not actually downloaded. Remember, spark is lazy. Instead, a link to the url is stored in the spark session object so that the file can be downloaded when we need to perform an operation on it.

Later, we will tell spark to return a handle to a dataframe consisting of the data contained in this url. At that point, lazy spark, will download the data file to each worker (at their root), open the file, and distribute the data accross the cluster. We will see proof of this when we call the addFile() function. Our server logs will show that the workers are making web requests to it.

## 4.1. Add the file using Spark Context

In [10]:
ip_address = spark_helper.determine_ip_address()
csv_file_name = "nasdaq_2019.csv"
csv_file_url = "http://{0}:{1}/{2}".format(ip_address, web_server_port, csv_file_name)
print("Uploading file '{0}' to Spark cluster.".format(csv_file_url))
sc.addFile(csv_file_url)

 * Serving Flask app 'PythonHttpFileServer' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://15.4.12.12:80/ (Press CTRL+C to quit)
INFO:root:Get /ml-training-jupyter-notebooks/Example Data Sets/nasdaq_2019.csv


Uploading file 'http://15.4.12.12:80/nasdaq_2019.csv' to Spark cluster.


INFO:werkzeug:15.4.12.12 - - [15/Jan/2022 21:32:31] "GET /nasdaq_2019.csv HTTP/1.1" 200 -


## 4.2. Use koalas to open the file on spark

Import the utility function to convert a date string to a datetime object from our utilities module

In [11]:
# Import the utilities module we wrote
import importlib.util
spec = importlib.util.spec_from_file_location("utilities", "../../../Utilities/utilities.py")
utilities = importlib.util.module_from_spec(spec)
spec.loader.exec_module(utilities)

# Define a mapping to convert our data field to the correct type
converter_mapping = {
    "date": utilities.convert_date_string_to_date
}

Load our OHCLV data Into a koalas dataframe and pull out a single day in the say way we would in pandas

In [12]:
# Avoid a warning
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

from databricks import koalas
koalas_dataframe = koalas.read_csv(u"file:////nasdaq_2019.csv", converters=converter_mapping)

INFO:spark:Patching spark automatically. You can disable it by setting SPARK_KOALAS_AUTOPATCH=false in your environment
INFO:root:Get /ml-training-jupyter-notebooks/Example Data Sets/nasdaq_2019.csv
INFO:werkzeug:15.4.7.101 - - [15/Jan/2022 21:32:40] "GET /nasdaq_2019.csv HTTP/1.1" 200 -
INFO:root:Get /ml-training-jupyter-notebooks/Example Data Sets/nasdaq_2019.csv
INFO:root:Get /ml-training-jupyter-notebooks/Example Data Sets/nasdaq_2019.csv
INFO:werkzeug:15.4.7.103 - - [15/Jan/2022 21:32:44] "GET /nasdaq_2019.csv HTTP/1.1" 200 -
INFO:werkzeug:15.4.7.102 - - [15/Jan/2022 21:32:44] "GET /nasdaq_2019.csv HTTP/1.1" 200 -


We should see the workers download the file in the logs. If we log into the nodes we can see the file is located on the filesystem root.

In [13]:
! kubectl -n spark get pods

NAME                                            READY     STATUS    RESTARTS   AGE
spark-jupyter-sklearn-03a4877e5fa79c97-exec-1   1/1       Running   0          49s
spark-jupyter-sklearn-03a4877e5fa79c97-exec-2   1/1       Running   0          49s
spark-jupyter-sklearn-03a4877e5fa79c97-exec-3   1/1       Running   0          49s


In [14]:
! kubectl -n spark exec -ti spark-jupyter-sklearn-03a4877e5fa79c97-exec-1 -- find / -name nasdaq_2019.csv

/nasdaq_2019.csv


With the data loaded into a koalas dataframe we can access the data in the same way we would from a pandas dataframe

In [15]:
koalas_dataframe.head()

,ticker,interval,date,open,high,low,close,volume
0,AABA,D,2019-07-01,70.90,71.5200,70.3250,70.57,10234800
1,AAL,D,2019-07-01,33.14,33.6632,32.5301,32.88,8995100
2,AAME,D,2019-07-01,2.43,2.4300,2.4000,2.40,500
3,AAOI,D,2019-07-01,10.70,10.8900,10.0100,10.18,883100
4,AAON,D,2019-07-01,50.57,50.9850,48.5600,49.73,180200


In [17]:
# Sort based on the date column
koalas_dataframe = koalas_dataframe.sort_values("date")
df_01_01_2019 = koalas_dataframe.loc[koalas_dataframe["date"] == '2019-01-01']
df_01_01_2019.head()

,ticker,interval,date,open,high,low,close,volume
93620,AABA,D,2019-01-01,57.94,57.94,57.94,57.94,0
93621,AAL,D,2019-01-01,32.11,32.11,32.11,32.11,0
93622,AAME,D,2019-01-01,2.41,2.41,2.41,2.41,0
93623,AAOI,D,2019-01-01,15.43,15.43,15.43,15.43,0
93624,AAON,D,2019-01-01,35.06,35.06,35.06,35.06,0


# 5. Prepare Worker Nodes

In order for us to be able to execute code on our spark cluster, we need to install relevant python libraries on the worker nodes. If you do not, you might see an error as follows when asking spark to do something:
```
PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/pyspark/worker.py", line 588, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/usr/local/lib/python3.6/site-packages/pyspark/worker.py", line 421, in read_udfs
    arg_offsets, f = read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=0)
  File "/usr/local/lib/python3.6/site-packages/pyspark/worker.py", line 249, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/usr/local/lib/python3.6/site-packages/pyspark/worker.py", line 69, in read_command
    command = serializer._read_with_length(file)
  File "/usr/local/lib/python3.6/site-packages/pyspark/serializers.py", line 160, in _read_with_length
    return self.loads(obj)
  File "/usr/local/lib/python3.6/site-packages/pyspark/serializers.py", line 430, in loads
    return pickle.loads(obj, encoding=encoding)
  File "/usr/local/lib/python3.6/site-packages/pyspark/cloudpickle/cloudpickle.py", line 562, in subimport
    __import__(name)
ModuleNotFoundError: No module named 'pandas'
```

Dont be confused, even though the module is installed locally, it does not mean it is installed on the work.

For the purposes of this notebook, we will be using pandas, numpy, koalas, scikit-learn, sklearn. Luclikly they have already been installed on the docker image we are using for our worker (configured when we created the spark session). 

If you are unsure of what is installed on your workers, we can log into each worker pod in our kubernetes cluster and execute shell commands to interrogate or even install things.

In [18]:
! kubectl -n spark get pod

NAME                                            READY     STATUS    RESTARTS   AGE
spark-jupyter-sklearn-03a4877e5fa79c97-exec-1   1/1       Running   0          1m
spark-jupyter-sklearn-03a4877e5fa79c97-exec-2   1/1       Running   0          1m
spark-jupyter-sklearn-03a4877e5fa79c97-exec-3   1/1       Running   0          1m


In [19]:
! kubectl -n spark exec -ti spark-jupyter-sklearn-03a4877e5fa79c97-exec-1 -- pip3 list

Package         Version
--------------- -------
click           8.0.3
cloudpickle     2.0.0
cycler          0.11.0
findspark       1.4.2
Flask           2.0.2
fonttools       4.28.5
future          0.18.2
hyperopt        0.2.7
itsdangerous    2.0.1
Jinja2          3.0.3
joblib          1.1.0
kiwisolver      1.3.2
kneed           0.7.0
koalas          0.32.0
MarkupSafe      2.0.1
matplotlib      3.5.1
netifaces       0.11.0
networkx        2.6.3
numpy           1.22.0
packaging       21.3
pandas          1.3.5
Pillow          9.0.0
pip             21.3.1
progressbar     2.5
py4j            0.10.9
pyarrow         6.0.1
pyparsing       3.0.6
pyspark         3.1.1
python-dateutil 2.8.2
pytz            2021.3
scikit-learn    1.0.2
scipy           1.7.3
setuptools      49.2.1
six             1.16.0
sklearn         0.0
threadpoolctl   3.0.0
tqdm            4.62.3
Werkzeug        2.0.2


In [20]:
! kubectl -n spark exec -ti spark-jupyter-sklearn-03a4877e5fa79c97-exec-1 -- pip3 install pandas

# 5. Submit Python Code To Spark Cluster

In this section of the notebook we are going to apply the kmeans algorithm from sklearn to each date in our koalas_dataframe object.
To do this, we are going to write a function that applies the algorithm to a dataframe; the assumption being the dataframe only contains data related to the same date.
Note: Most of this is a review and reworking of the content contained in 
<a href="../Cluster%20Analysis/K-Means.ipynb">Cluster Analysis/K-Means.ipynb</a>.

We create our data frame for testing based on a subset of our real data.

In [21]:
df_01_01_2019.head()

,ticker,interval,date,open,high,low,close,volume
93620,AABA,D,2019-01-01,57.94,57.94,57.94,57.94,0
93621,AAL,D,2019-01-01,32.11,32.11,32.11,32.11,0
93622,AAME,D,2019-01-01,2.41,2.41,2.41,2.41,0
93623,AAOI,D,2019-01-01,15.43,15.43,15.43,15.43,0
93624,AAON,D,2019-01-01,35.06,35.06,35.06,35.06,0


We then write and test our function

In [25]:
koalas.set_option('compute.ops_on_diff_frames', True)

In [22]:
from sklearn.cluster import KMeans
model = KMeans(n_clusters=5, random_state=42)

In [26]:
def perform_kmeans_on_dataframe(df, column_names=["open"]):
    
    # Create a copy of our dataframe so we can play around
    tmp = df.copy()
    columns = column_names.copy()

    # IF we only supplied one column name, we will need to create a bogus column
    # Kmeans requires a 2D array so we will add a static column
    bogus_column = None
    if len(column_names) < 2:
        bogus_column = "Y"
        tmp[bogus_column] = [1 for x in range(0, tmp.shape[0])]
        columns.append(bogus_column)
        
    # Set the parameters for our model
    # It expects a 2D array where the columns are our features
    model_parameters = tmp[[*columns]].to_numpy()
    
    # Train the model
    trained_model = model.fit(model_parameters)
    
    # Extract the information
    cluster_indices = trained_model.labels_.astype(int).tolist()
    if bogus_column in columns:
        cluster_centroids = [trained_model.cluster_centers_[i][0] for i in cluster_indices]
    else:
        cluster_centroids = [str(trained_model.cluster_centers_[i].tolist()) for i in cluster_indices]
        
    # Update the dataframe (setting special options to allow koalas to work)
#    option_value = koalas.get_option('compute.ops_on_diff_frames')
    #koalas.set_option('compute.ops_on_diff_frames', True)

    tmp["cluster_centroids"] = koalas.Series(cluster_centroids, index=tmp.index.to_numpy())
    tmp["cluster_indices"] = koalas.Series(cluster_indices, index=tmp.index.to_numpy())

#    koalas.set_option('compute.ops_on_diff_frames', option_value)
    
    # Determine which columns we want to return
    columns = tmp.columns.to_list()
    if bogus_column in columns:
        columns.remove(bogus_column)
    
    return tmp[[*columns]]


In [27]:
perform_kmeans_on_dataframe(df_01_01_2019, column_names=["open", "close"]).head()

,ticker,interval,date,open,high,low,close,volume,cluster_centroids,cluster_indices
93816,APPF,D,2019-01-01,59.22,59.22,59.22,59.22,0,"[67.53273664825046, 67.53273664825046]",0
94988,HMST,D,2019-01-01,21.23,21.23,21.23,21.23,0,"[13.056601167315154, 13.056601167315154]",2
95031,HYGS,D,2019-01-01,5.00,5.00,5.00,5.00,0,"[13.056601167315154, 13.056601167315154]",2
95149,IRIX,D,2019-01-01,4.70,4.70,4.70,4.70,0,"[13.056601167315154, 13.056601167315154]",2
95284,LBTYB,D,2019-01-01,21.00,21.00,21.00,21.00,0,"[13.056601167315154, 13.056601167315154]",2


In [28]:
from datetime import datetime
now = datetime.now()
result = df_01_01_2019.groupby("date").apply(perform_kmeans_on_dataframe, column_names=["open"])
later = datetime.now()
difference = (later - now).total_seconds()
print("total calculation time: {0}s".format(difference))
result.head()

TypeError: apply() got an unexpected keyword argument 'column_names'

We can now run this function against out entire dataframe.

In [ ]:
from datetime import datetime
now = datetime.now()
result = koalas_dataframe.groupby("date").apply(perform_kmeans_on_dataframe, column_names=["open"])
later = datetime.now()
difference = (later - now).total_seconds()
print("total calculation time: {0}s".format(difference))
result.head()

We see that computing every date is almost as fast as computing a single date.

# 6. Cleanup Spark Cluster On Kubernetes

In [ ]:
import os
if os.path.exists(csv_link_path) and os.path.islink(csv_link_path):
    os.unlink(csv_link_path)
    print("Deleted symlinked data file")

In [ ]:
sc.stop()

In [ ]:
! kubectl -n spark get pod